In [1]:

###############################################################################
# Random Forest Classification Model (TensorFlow)                             #
# Based on the Implementation of:                                             #
# For GoldenEye Dataset                                                       #
# https://www.tensorflow.org/decision_forests/tutorials/beginner_colab        #
###############################################################################


In [2]:
# Installieren aller benötigten Pakete
!pip install pandas
!pip install tensorflow_decision_forests
!pip install wurlitzer
!pip install matplotlib
!pip install ipython

In [9]:
# Laden der benötigten Python Pakete
import pandas as pd
import numpy as np
import tensorflow_decision_forests as tfdf
from wurlitzer import sys_pipes
import matplotlib.pyplot as plt

In [4]:
# Prüfung der installierten TensorFlow Decision Forests Version
print(f"Found TensorFlow Decision Forests v{tfdf.__version__}")


Found TensorFlow Decision Forests v0.1.7


In [6]:
# Laden der Netzwerk Traffic Daten für den GoldenEye Angriff
data_GoldenEye = pd.read_csv('../Data/Thursday-15-02-2018_GoldenEye-Attack.csv')
# Umbenennen der einzelnen Spalte für eine bessere Kompatibilität mit TensorFlow
data_GoldenEye.rename(columns={
    'Bwd Pkt Len Std':'bwd_pkt_len_std',
    'Flow IAT Min':'flow_iat_min',
    'Fwd IAT Min':'fwd_iat_min',
    'Flow IAT Mean':'flow_iat_mean',
    'Label':'label'
},
inplace=True)

In [7]:
# Festlegen des Wertes der bestimmten Variable
label = 'label'

In [10]:
# Aufteilen des Datasets in Training- und Test-Daten
def split_dataset(dataset,  test_ratio=0.30):
    """Splits a panda dataframe in two dataframes."""
    test_indices = np.random.rand(len(dataset)) < test_ratio
    return dataset[~test_indices], dataset[test_indices]

training_data_GoldenEye, testing_data_GoldenEye = split_dataset(data_GoldenEye)

print("{} examples in training, {} examples for testing.".format(
    len(training_data_GoldenEye), len(testing_data_GoldenEye)))


726193 examples in training, 311392 examples for testing.


In [11]:
# Konvertieren des Panda Dataframes in ein TensorFlow Dataset
print("Converting Panda Dataframe into TensorFlow Dataset...")
training_dataset_GoldenEye = tfdf.keras.pd_dataframe_to_tf_dataset(training_data_GoldenEye, label=label)
testing_dataset_GoldenEye = tfdf.keras.pd_dataframe_to_tf_dataset(testing_data_GoldenEye, label=label)


Converting Panda Dataframe into TensorFlow Dataset...


In [12]:
# Erstellen des Random Forest Modells
model = tfdf.keras.RandomForestModel()
model.compile(metrics=["accuracy"])


In [13]:
# Trainieren des Modells
print("Training the Model: ")
with sys_pipes():
    model.fit(x=training_dataset_GoldenEye)

Training the Model: 
2021-06-28 10:19:37.125291: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-06-28 10:19:37.150848: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2199995000 Hz
11347/11347 [==============================] - 11s 738us/step
[INFO kernel.cc:746] Start Yggdrasil model training
[INFO kernel.cc:747] Collect training examples
[INFO kernel.cc:392] Number of batches: 11347
[INFO kernel.cc:393] Number of examples: 726193
[INFO kernel.cc:769] Dataset:
Number of records: 726193
Number of columns: 5

Number of columns by type:
	NUMERICAL: 4 (80%)
	CATEGORICAL: 1 (20%)

Columns:

NUMERICAL: 4 (80%)
	0: "bwd_pkt_len_std" NUMERICAL mean:121.772 min:0 max:931.26 sd:207.567
	1: "flow_iat_mean" NUMERICAL mean:2.9511e+06 min:0 max:1.19992e+08 sd:1.12061e+07
	2: "flow_iat_min" NUMERICAL mean:2.37826e+06 min:0 max:1.19992e+08 sd:1.10505e+07
	3: "fwd_iat_min" NUMERICAL mean:2.5

In [14]:
# Evaluation des trainierten Modells mit den Testdaten
print("Evaluating the Model...")
evaluation = model.evaluate(testing_dataset_GoldenEye, return_dict=True)

print()

for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")

Evaluating the Model...
4866/4866 [==============================] - 14s 3ms/step - loss: 0.0000e+00 - accuracy: 0.9987

loss: 0.0000
accuracy: 0.9987


In [15]:
# Trainiertes Modell für die spätere Verwendung abspeichern
model.save("../Data/Models/goldeneye_model")


INFO:tensorflow:Assets written to: ../Data/Models/goldeneye_model/assets
INFO:tensorflow:Assets written to: ../Data/Models/goldeneye_model/assets


In [16]:
# Plotten des ersten Baumes innerhalb des Decision Forests
with open('../Data/Models/GoldenEye_Model_Tree.html', 'w') as f:
    f.write(tfdf.model_plotter.plot_model(model, tree_idx=0, max_depth=3))
tfdf.model_plotter.plot_model(model, tree_idx=0, max_depth=3)


'\n<script src="https://d3js.org/d3.v6.min.js"></script>\n<div id="tree_plot_719bfc1645154bf9a09a4add8f6df17e"></div>\n<script>\n/*\n * Copyright 2021 Google LLC.\n * Licensed under the Apache License, Version 2.0 (the "License");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *     https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an "AS IS" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\n/**\n *  Plotting of decision trees generated by TF-DF.\n *\n *  A tree is a recursive structure of node objects.\n *  A node contains one or more of the following components:\n *\n *    - A value: Representing the output of the node. If the node is not a leaf,\

In [17]:
# Erstellen einer Bilanz für das trainierte Modell
model.summary()

Model: "random_forest_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (4):
	bwd_pkt_len_std
	flow_iat_mean
	flow_iat_min
	fwd_iat_min

No weights

Variable Importance: NUM_NODES:
    1.   "flow_iat_mean" 46522.000000 ################
    2.     "fwd_iat_min" 39806.000000 ###########
    3.    "flow_iat_min" 38528.000000 ###########
    4. "bwd_pkt_len_std" 19735.000000 

Variable Importance: NUM_AS_ROOT:
    1.   "flow_iat_mean" 233.000000 ################
    2.     "fwd_iat_min" 41.000000 #
    3. "bwd_pkt_len_std" 26.000000 

Variable Importance: SUM_SCORE:
    1.   "flow_iat_mean" 13141977.443992 ################
    2.    "flow_iat_min" 10991524.746150 ############
    3. "bwd_pkt_len_std" 747

In [19]:
# Erstellen von Grafiken für die Effizienz des Trainings
logs = model.make_inspector().training_logs()
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot([log.num_trees for log in logs], [log.evaluation.accuracy for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Accuracy (out-of-bag)")

plt.subplot(1,2,2)
plt.plot([log.num_trees for log in logs], [log.evaluation.loss for log in logs])
plt.xlabel("Number of trees")
plt.ylabel("Logloss (out-of-bag)")

plt.savefig('../Data/Visualized/GoldenEye_Model.png')
plt.clf()

<Figure size 864x288 with 0 Axes>